In [146]:
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.neighbors import KNeighborsClassifier
import random
import matplotlib.patheffects as PathEffects

In [147]:
from keras.layers import Input, Conv2D, Conv1D, Lambda, Dense, Flatten,MaxPooling2D, concatenate
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras import backend as K
# from tensorflow.keras.optimizers import SGD
from keras.losses import binary_crossentropy
import os
import pickle
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam

In [148]:
from itertools import permutations

In [149]:
import seaborn as sns

In [150]:
from keras.datasets import mnist
from sklearn.manifold import TSNE

In [151]:
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf

In [152]:
#(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [153]:
def get_train_test_dataset(train_test_ratio):
    X = []
    y = []
    X = pd.read_excel(io='CompareResult4.xlsx', sheet_name='Sheet1',usecols="J,K,L")
    X.fillna(0,inplace=True)
    X=X.to_numpy()
    return X

In [154]:
x_train = get_train_test_dataset(0.5)


In [155]:
data_size=8396
#x_train_flat_anchor = tf.reshape(x_train[0:,0], [data_size, 1])

x_train_flat_anchor = x_train[0:,0]
x_train_flat_positive = x_train[0:,1]
x_train_flat_negative = x_train[0:,2]

#print(x_train_flat_anchor)
#x_test_flat = x_test.reshape(-1,784)

In [156]:
#tsne = TSNE()
#train_tsne_embeds = tsne.fit_transform(x_train_flat[:512])
#scatter(train_tsne_embeds, y_train[:512], "Samples from Training Data")

#eval_tsne_embeds = tsne.fit_transform(x_test_flat[:512])
#scatter(eval_tsne_embeds, y_test[:512], "Samples from Validation Data")

In [157]:
Classifier_input = Input((784,))
Classifier_output = Dense(10, activation='softmax')(Classifier_input)
Classifier_model = Model(Classifier_input, Classifier_output)

In [158]:
from sklearn.preprocessing import LabelBinarizer

In [159]:
le = LabelBinarizer()

In [160]:
#y_train_onehot = le.fit_transform(y_train)
#y_test_onehot = le.transform(y_test)

In [161]:
Classifier_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [162]:
#Classifier_model.fit(x_train_flat,y_train_onehot, validation_data=(x_test_flat,y_test_onehot),epochs=10)

In [163]:
def generate_triplet():
    x= get_train_test_dataset(0.7);  
                
    return x;

In [164]:
X_train = generate_triplet()

In [165]:
print(X_train.shape)

(8396, 3)
(8396, 3)


## Triplet NN

In [166]:
def create_base_network(in_dims):
    """
    Base network to be shared.
    """
    model = Sequential()

    model.add(Flatten(input_shape=(1,1))) #this line
    #model.add(Conv1D(1,(7,7),padding='same',input_shape=(in_dims[0],in_dims[1],in_dims[2],),activation='relu',name='conv1'))
    #model.add(MaxPooling2D((2,2),(2,2),padding='same',name='pool1'))
    #model.add(Conv1D(10,(1),padding='same',activation='relu',name='conv2'))
    #model.add(MaxPooling2D((2,2),(2,2),padding='same',name='pool2'))
    #model.add(Flatten(name='flatten'))
    model.add(Dense(10,name='embeddings'))
    # model.add(Dense(600))
    return model

In [167]:
adam_optim = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999)

C:\Users\ali.nizam\AppData\Roaming\Python\Python310\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
C:\Users\ali.nizam\AppData\Roaming\Python\Python310\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [168]:
data_size=1
anchor_input = Input((1, ), name='anchor_input')
positive_input = Input((1, ), name='positive_input')
negative_input = Input((1, ), name='negative_input')

# Shared embedding layer for positive and negative items
Shared_DNN = create_base_network([1,])


encoded_anchor = Shared_DNN(anchor_input)
encoded_positive = Shared_DNN(positive_input)
encoded_negative = Shared_DNN(negative_input)


merged_vector = concatenate([encoded_anchor, encoded_positive, encoded_negative], axis=-1, name='merged_layer')
print(merged_vector.shape)
model = Model(inputs=[anchor_input,positive_input, negative_input], outputs=merged_vector)




(None, 30)
(None, 30)


In [169]:
#from tensorflow import keras
#loss_fn = keras.losses.SparseCategoricalCrossentropy()
#model.compile(loss=loss_fn, optimizer=adam_optim)

In [170]:
model.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 anchor_input (InputLayer)      [(None, 1)]          0           []                               
                                                                                                  
 positive_input (InputLayer)    [(None, 1)]          0           []                               
                                                                                                  
 negative_input (InputLayer)    [(None, 1)]          0           []                               
                                                                                                  
 sequential_3 (Sequential)      (None, 10)           20          ['anchor_input[0][0]',           
                                                                  'positive_input[0][0]',  

In [171]:
Anchor = x_train[:,0]
Positive = x_train[:,1]
Negative = x_train[:,2]
Anchor_test = x_train[:,0]
Positive_test = x_train[:,1]
Negative_test = x_train[:,2]

Y_dummy = np.empty((Anchor.shape[0],1))
print(Y_dummy.size)
Y_dummy2 = np.empty((Anchor_test.shape[0],1))
print(Y_dummy2.size)

8396
8396
8396
8396


In [172]:
[Anchor,Positive,Negative]

[array([1, 1, 1, ..., 1, 1, 1], dtype=int64),
 array([2, 5, 5, ..., 2, 2, 2], dtype=int64),
 array([5, 5, 2, ..., 2, 2, 2], dtype=int64)]

[array([1, 1, 1, ..., 1, 1, 1], dtype=int64),
 array([2, 5, 5, ..., 2, 2, 2], dtype=int64),
 array([5, 5, 2, ..., 2, 2, 2], dtype=int64)]

In [173]:
def triplet_loss(y_true, y_pred, alpha = 0.4):
    """
    Implementation of the triplet loss function
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor data
            positive -- the encodings for the positive data (similar to anchor)
            negative -- the encodings for the negative data (different from anchor)
    Returns:
    loss -- real number, value of the loss
    """

    import sys
    print('y_pred.shape = ',y_pred)

    total_lenght = y_pred.shape.as_list()[-1]
#     print('total_lenght=',  total_lenght)
#     total_lenght =12


    anchor = y_pred[:,0:int(total_lenght*1/3)]
    positive = y_pred[:,int(total_lenght*1/3):int(total_lenght*2/3)]
    negative = y_pred[:,int(total_lenght*2/3):int(total_lenght*3/3)]


    # distance between the anchor and the positive
    pos_dist = K.sum(K.square(anchor-positive),axis=1)

    # distance between the anchor and the negative
    neg_dist = K.sum(K.square(anchor-negative),axis=1)
    # compute loss
    basic_loss = pos_dist-neg_dist+alpha

    loss = K.maximum(basic_loss,0.0)

    return loss

model.compile(loss=triplet_loss, optimizer=adam_optim)


model.fit([Anchor,Positive,Negative],y=Y_dummy,validation_data=([Anchor_test,Positive_test,Negative_test],Y_dummy2), batch_size=10, epochs=10)

Epoch 1/10
y_pred.shape =  Tensor("model_10/merged_layer/concat:0", shape=(None, 30), dtype=float32)
y_pred.shape =  Tensor("model_10/merged_layer/concat:0", shape=(None, 30), dtype=float32)
835/840 [============================>.] - ETA: 0s - loss: 0.4033y_pred.shape =  Tensor("model_10/merged_layer/concat:0", shape=(None, 30), dtype=float32)


C:\Users\ali.nizam\AppData\Roaming\Python\Python310\site-packages\tensorflow\python\framework\constant_op.py:102: RuntimeWarning: overflow encountered in cast
  return ops.EagerTensor(value, ctx.device_name, dtype)


840/840 [==============================] - 1s 1ms/step - loss: 0.4033 - val_loss: 0.4033
Epoch 2/10
840/840 [==============================] - 1s 1ms/step - loss: 0.4033 - val_loss: 0.4032
Epoch 3/10
840/840 [==============================] - 1s 1ms/step - loss: 0.4032 - val_loss: 0.4032
Epoch 4/10
840/840 [==============================] - 1s 1ms/step - loss: 0.4032 - val_loss: 0.4032
Epoch 5/10
840/840 [==============================] - 1s 1ms/step - loss: 0.4032 - val_loss: 0.4031
Epoch 6/10
840/840 [==============================] - 1s 1ms/step - loss: 0.4031 - val_loss: 0.4031
Epoch 7/10
840/840 [==============================] - 1s 2ms/step - loss: 0.4031 - val_loss: 0.4031
Epoch 8/10
840/840 [==============================] - 1s 1ms/step - loss: 0.4031 - val_loss: 0.4030
Epoch 9/10
840/840 [==============================] - 1s 1ms/step - loss: 0.4030 - val_loss: 0.4030
Epoch 10/10
840/840 [==============================] - 1s 1ms/step - loss: 0.4030 - val_loss: 0.4030


Epoch 1/10
y_pred.shape =  Tensor("model_13/merged_layer/concat:0", shape=(None, 30), dtype=float32)
y_pred.shape =  Tensor("model_13/merged_layer/concat:0", shape=(None, 30), dtype=float32)
840/840 [==============================] - 1s 1ms/step - loss: 0.4050 - val_loss: 0.4049
Epoch 2/10
840/840 [==============================] - 1s 1ms/step - loss: 0.4049 - val_loss: 0.4049
Epoch 3/10
840/840 [==============================] - 1s 1ms/step - loss: 0.4049 - val_loss: 0.4049
Epoch 4/10
840/840 [==============================] - 1s 1ms/step - loss: 0.4049 - val_loss: 0.4048
Epoch 5/10
840/840 [==============================] - 1s 1ms/step - loss: 0.4048 - val_loss: 0.4048
Epoch 6/10
840/840 [==============================] - 1s 1ms/step - loss: 0.4048 - val_loss: 0.4047
Epoch 7/10
840/840 [==============================] - 1s 1ms/step - loss: 0.4047 - val_loss: 0.4047
Epoch 8/10
840/840 [==============================] - 1s 1ms/step - loss: 0.4047 - val_loss: 0.4047
Epoch 9/10
840/840 [

In [174]:
trained_model = Model(inputs=anchor_input, outputs=encoded_anchor)

In [175]:
# trained_model.load_weights('triplet_model_MNIST.hdf5')

In [176]:
tsne = TSNE()
X_train_trm = trained_model.predict(x_train[:512].reshape(-1,1))
# X_test_trm = trained_model.predict(x_test[:512].reshape(-1,28,28,1))
train_tsne_embeds = tsne.fit_transform(X_train_trm)
# eval_tsne_embeds = tsne.fit_transform(X_test_trm)

C:\Users\ali.nizam\AppData\Roaming\Python\Python310\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
C:\Users\ali.nizam\AppData\Roaming\Python\Python310\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
C:\Users\ali.nizam\AppData\Roaming\Python\Python310\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
C:\Users\ali.nizam\AppData\Roaming\Python\Python310\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [177]:
train_tsne_embeds

array([[  0.05170216, -21.117847  ],
       [  0.35083055,  11.7853985 ],
       [ -0.7663615 ,   7.741275  ],
       ...,
       [  0.05162948, -21.134188  ],
       [  0.17648223,   8.566288  ],
       [ -0.55646175,   9.458549  ]], dtype=float32)

array([[-21.036894  ,  -4.4034395 ],
       [  9.478342  ,   2.2317417 ],
       [  4.4925876 ,   2.7020884 ],
       ...,
       [-21.050306  ,  -4.4068284 ],
       [ 10.854581  ,  -0.21599801],
       [  8.0583    ,   3.4296262 ]], dtype=float32)

In [178]:
# scatter(train_tsne_embeds, y_train[:512], "Training Data After TNN")

In [179]:
# scatter(eval_tsne_embeds, y_test[:512], "Validation Data After TNN")

In [180]:
# X_train_trm = trained_model.predict(x_train.reshape(-1,28,28,1))
# X_test_trm = trained_model.predict(x_test.reshape(-1,28,28,1))
#
# Classifier_input = Input((4,))
# Classifier_output = Dense(10, activation='softmax')(Classifier_input)
# Classifier_model = Model(Classifier_input, Classifier_output)
#
#
# Classifier_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
#
# Classifier_model.fit(X_train_trm,y_train_onehot, validation_data=(X_test_trm,y_test_onehot),epochs=10)